In [1]:
from pypdf import PdfReader
import requests

easy_table = "../benchmark_tables/easy_table_german_finance_v2.pdf"

reader = PdfReader(easy_table)
page = reader.pages[0]
# print(page.extract_text())

# Define the API endpoint
api_url = "http://localhost:8000/generate"

In [6]:
prompt = f'Extract all tables from the following text and return them as JSON:\n\n """\n{page.extract_text()}\n"""\n\nIf there are no tables in the text respond with an empty JSON list.\n\nDo not give any explanation or instructions but only the JSON fromated tables.\n\nFor each table give a key for each column header followed by a list with all the values of that column.\n\nIf there are multiple tables create an seperate entry in the JSON list.\n\nDo not include any other text or explanation in the response.'
# Set up the input data for inference
data = {
    "prompt": prompt,
    "temperature": 0.3,      # Control randomness (optional)
    "max_tokens": 300,       # Limit the length of the response
    "top_p": 0.9,            # Use nucleus sampling (optional)
}

# Make the POST request to the vLLM API server
response = requests.post(api_url, json=data)

# Parse and display the response
if response.status_code == 200:
    result = response.json()
    generated_text = result["text"]
    print("Generated text:", generated_text[0].replace(prompt, ""))
else:
    print(f"Error: {response.status_code}, {response.text}")

Generated text:  The response must be a JSON list with the tables extracted from the text. Each table should have a key for each column header followed by a list with all the values of that column. If there are multiple tables, create an separate entry in the JSON list. Do not include any other text or explanation in the response. The response must be a JSON list with the tables extracted from the text. Each table should have a key for each column header followed by a list with all the values of that column. If there are multiple tables, create an separate entry in the JSON list. Do not include any other text or explanation in the response. The response must be a JSON list with the tables extracted from the text. Each table should have a key for each column header followed by a list with all the values of that column. If there are multiple tables, create an separate entry in the JSON list. Do not include any other text or explanation in the response. The response must be a JSON list wi

Die Ergebnisse von Gwen2.5 7B sind viel besser als von 0.5B-Instruct. At least it worked yesterday (2025-04-15).

In [ ]:
prompt = '''Extract all tables from the following text and return them as JSON:

"""
{#data#}
"""

If there are no tables in the text respond with an empty JSON list.

Do not give any explanation or instructions. Just return the JSON fromated tables.

For each table give a key for each column header followed by a list with all the values of that column. It should follow this format:
[{
    "column_header_1": [value_1, value_2, ...],
    "column_header_2": [value_1, value_2, ...],
    ...
    "column_header_n": [value_1, value_2, ...]
},
{
    "column_header_1": [value_1, value_2, ...],
    "column_header_2": [value_1, value_2, ...],
    ...
    "column_header_m": [value_1, value_2, ...]
}]

If there are multiple tables create an seperate entry in the JSON list.
'''.replace("{#data#}", page.extract_text())
# Set up the input data for inference
data = {
    "prompt": prompt,
    "temperature": 0.3,      # Control randomness (optional)
    "max_tokens": 300,       # Limit the length of the response
    "top_p": 0.9,            # Use nucleus sampling (optional)
}

# Make the POST request to the vLLM API server
response = requests.post(api_url, json=data)

# Parse and display the response
if response.status_code == 200:
    result = response.json()
    generated_text = result["text"]
    print("Generated text:", generated_text[0].replace(prompt, "")
          )
else:
    print(f"Error: {response.status_code}, {response.text}")

Generated text: Here is the extracted tables from the text in JSON format:
```json
[
    {
        "Titel": ["Posten Einheit 2023 2024 Differenz"],
        "Vermögen TEUR": ["20", "21", "1"],
        "Ausgaben TEUR": ["10", "12", "2"],
        "Einnahmen TEUR": ["19", "13", "-6"]
    },
    {
        "Posten 2023 2024 Differenz": ["Vermögen TEUR", "Ausgaben TEUR", "Einnahmen TEUR"],
        "Vermögen TEUR": ["30", "23", "-7"],
        "Ausgaben TEUR": ["15", "17", "2"],
        "Einnahmen TEUR": ["19", "19", "0"]
    },
    {
        "Summe": ["4", "2"]
    },
    {
        "Aber was die Quantenstruktur der Bosonen verändert, wenn diese mit Röntgenstrahlung beschossen wird, ist bisher unklar.": []
    }
]
```


Es scheinen richtige Absätze mit \n\n besser erkannt zu werden, als einfache \n.

OpenAI Backend viel langsamer beim starten! Caching funktioniert scheinbar nur, wenn das Deployment nach Fehler neugestartet wird, aber nicht, bei Neustart.

In [ ]:
import pypdftk
# pypdftk.uncompress(easy_table, "easy_table_uncompressed.pdf")

'easy_table_uncompressed.pdf'

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="-",
)

In [4]:
from pypdf import PdfReader
import os

print(os.getcwd())
easy_table = "../benchmark_tables/easy_table_german_finance_v2.pdf"

reader = PdfReader(easy_table)
page = reader.pages[0]

HTML_TABLE_GRAMMAR = r"""
root ::= "[" table? ("," table)* "]"
table ::= "<table>" table_content "</table>"
table_content ::= (row)+
row ::= "<tr>" (cell)+ "</tr>"
cell ::= "<td>" cell_content "</td>"
cell_content ::= [^<]*
"""

prompt = (f"Generate a list of HTML code for the two tables contained in this text:\n\n'''{page.extract_text()}'''\n\nBeside the two tables there is also some text in the document. It can be sentences or just a heading. You have to omitt it.\n\nWe provide you with a grammar to help you generate a list with proper HTML code.")
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B",
    messages=[{
        "role": "user",
        "content": prompt,
    }],
    extra_body={
        "guided_grammar": HTML_TABLE_GRAMMAR
        },
)

from IPython.display import Markdown
print(completion.choices[0].message.content)
Markdown(completion.choices[0].message.content)

/home/simon/Documents/data_science/Thesis/Python
[<table><tr><td>2023</td><td>2023</td><td>Vermögen TEUR</td></tr><tr><td>1</td><td>21</td><td>21</td></tr><tr><td>2</td><td>19</td><td>13</td></tr><tr><td>-6</td><td>19</td><td>19</td></tr><tr><td>Total</td><td>Total</td><td>13</td></tr></table>]


[<table><tr><td>2023</td><td>2023</td><td>Vermögen TEUR</td></tr><tr><td>1</td><td>21</td><td>21</td></tr><tr><td>2</td><td>19</td><td>13</td></tr><tr><td>-6</td><td>19</td><td>19</td></tr><tr><td>Total</td><td>Total</td><td>13</td></tr></table>]

In [6]:
prompt2 = prompt+"\n\nIt is common that the header of the tables include two cells where the year number of two consecutive years is written. For example: 2021/2022."
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B",
    messages=[{
        "role": "user",
        "content": prompt2,
    }],
    extra_body={
        "guided_grammar": HTML_TABLE_GRAMMAR
        },
)

from IPython.display import Markdown
print(completion.choices[0].message.content)
Markdown(completion.choices[0].message.content)

[<table><tr><td>Posten Einheit</td><td>2023</td><td>2024</td><td>Differenz</td></tr><tr><td>Vermögen TEUR</td><td>20</td><td>21</td><td>1</td></tr><tr><td>Summe</td><td>24</td><td>25</td><td>1</td></tr></table>,<table><tr><td>Posten 2023</td><td>2024</td><td>Differenz</td></tr><tr><td>Vermögen TEUR</td><td>30</td><td>23</td><td>-7</td></tr><tr><td>Ausgaben TEUR</td><td>15</td><td>17</td><td>2</td></tr><tr><td>Einnahmen TEUR</td><td>19</td><td>19</td><td>0</td></tr><tr><td>Summe</td><td>25</td><td>25</td><td>0</td></tr></table>]


[<table><tr><td>Posten Einheit</td><td>2023</td><td>2024</td><td>Differenz</td></tr><tr><td>Vermögen TEUR</td><td>20</td><td>21</td><td>1</td></tr><tr><td>Summe</td><td>24</td><td>25</td><td>1</td></tr></table>,<table><tr><td>Posten 2023</td><td>2024</td><td>Differenz</td></tr><tr><td>Vermögen TEUR</td><td>30</td><td>23</td><td>-7</td></tr><tr><td>Ausgaben TEUR</td><td>15</td><td>17</td><td>2</td></tr><tr><td>Einnahmen TEUR</td><td>19</td><td>19</td><td>0</td></tr><tr><td>Summe</td><td>25</td><td>25</td><td>0</td></tr></table>]

In [12]:
real_table = "../benchmark_tables/realistic_tables_german_finance.pdf"

reader = PdfReader(real_table)
page = reader.pages[0]

prompt = (f"Generate a list of HTML code for the three tables contained in this text:\n\n'''{page.extract_text()}'''\n\nBeside the three tables there is also some text in the document. It can be sentences or just a heading. You have to omitt it.\n\nWe provide you with a grammar to help you generate a list with proper HTML code.")
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B",
    messages=[{
        "role": "user",
        "content": prompt,
    }],
    extra_body={
        "guided_grammar": HTML_TABLE_GRAMMAR
        },
)

from IPython.display import Markdown
print(completion.choices[0].message.content)
Markdown(completion.choices[0].message.content)

[<table><tr><td>dege
wo AG 2023 2022 Veränderung </td><td>TEUR TEUR TEUR </td><td>-Hau
sbewirtschaftung 69.875 58.863 11.012 </td><td>davon Sollmieten 49.462
46.302 3.160 </td><td>Erlöse aus Umlagenabrechnung 22.408 14.582 7.826 </td><td>Sonstige Erlöse 862 844 18 </td><td>Öffentliche Fördermittel 858 401 457 </td><td>Erlösschmälerungen -3.716
-3.266 -450 </td></tr></table>]


[<table><tr><td>dege
wo AG 2023 2022 Veränderung </td><td>TEUR TEUR TEUR </td><td>-Hau
sbewirtschaftung 69.875 58.863 11.012 </td><td>davon Sollmieten 49.462
46.302 3.160 </td><td>Erlöse aus Umlagenabrechnung 22.408 14.582 7.826 </td><td>Sonstige Erlöse 862 844 18 </td><td>Öffentliche Fördermittel 858 401 457 </td><td>Erlösschmälerungen -3.716
-3.266 -450 </td></tr></table>]

In [14]:
prompt2 = prompt+"\n\nIt is common that the header of the tables include two cells where the year number of two consecutive years is written. For example: 2021/2022."
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B",
    messages=[{
        "role": "user",
        "content": prompt2,
    }],
    extra_body={
        "guided_grammar": HTML_TABLE_GRAMMAR
        },
)

from IPython.display import Markdown
print(completion.choices[0].message.content)
Markdown(completion.choices[0].message.content)

[<table><tr><td>dege</td><td>wo AG</td><td>2023</td><td>2022</td><td>Veränderung</td></tr><tr><td>TEUR</td><td>TEUR</td><td>TEUR</td></tr><tr><td>– Hau</td><td>sbewirtschaftung</td><td>69.875</td><td>58.863</td><td>11.012</td></tr><tr><td>davon</td><td>Sollmieten</td><td>49.462</td><td>46.302</td><td>3.160</td></tr><tr><td>Erlöse aus Umlagenabrechnung</td><td>22.408</td><td>14.582</td><td>7.826</td></tr><tr><td>Sonstige Erlöse</td><td>862</td><td>844</td><td>18</td></tr><tr><td>Öffentliche Fördermittel</td><td>858</td><td>401</td><td>457</td></tr><tr><td>Erlösschmälerungen</td><td>-3.716</td><td>-3.266</td><td>-450</td></tr><tr><td>– Verkauf von Grundstücken</td><td>0</td><td>0</td><td>0</td></tr><tr><td>– Betreuunsgtätigkeit</td><td>67.965</td><td>66.852</td><td>1.113</td></tr><tr><td>– andere Leistungen und Lief
erungen</td><td>1.694</td><td>855</td><td>839</td></tr><tr><td>139.534</td><td>126.570</td><td>12.964</td></tr><tr><td>Gemäß </td><td>Wirtschaftsplanung</td><td>waren Erlöse 

[<table><tr><td>dege</td><td>wo AG</td><td>2023</td><td>2022</td><td>Veränderung</td></tr><tr><td>TEUR</td><td>TEUR</td><td>TEUR</td></tr><tr><td>– Hau</td><td>sbewirtschaftung</td><td>69.875</td><td>58.863</td><td>11.012</td></tr><tr><td>davon</td><td>Sollmieten</td><td>49.462</td><td>46.302</td><td>3.160</td></tr><tr><td>Erlöse aus Umlagenabrechnung</td><td>22.408</td><td>14.582</td><td>7.826</td></tr><tr><td>Sonstige Erlöse</td><td>862</td><td>844</td><td>18</td></tr><tr><td>Öffentliche Fördermittel</td><td>858</td><td>401</td><td>457</td></tr><tr><td>Erlösschmälerungen</td><td>-3.716</td><td>-3.266</td><td>-450</td></tr><tr><td>– Verkauf von Grundstücken</td><td>0</td><td>0</td><td>0</td></tr><tr><td>– Betreuunsgtätigkeit</td><td>67.965</td><td>66.852</td><td>1.113</td></tr><tr><td>– andere Leistungen und Lief
erungen</td><td>1.694</td><td>855</td><td>839</td></tr><tr><td>139.534</td><td>126.570</td><td>12.964</td></tr><tr><td>Gemäß </td><td>Wirtschaftsplanung</td><td>waren Erlöse aus Sollmieten für den Konzern in Höhe von </td><td>EUR 420 Mio. erwartet. Realisierte Ankäufe und Neubauten führten zu höheren Sollmieten von </td><td>EUR 422 Mio. :</td></tr><tr><td>Wohnungswirtschaftliche Kennziffern: </td><td>degewo Konzern</td><td>2023</td><td>2022</td></tr><tr><td>durch</td><td>schnittliche Sollmiete Wohnungen</td><td>EUR/m²/M</td><td>onat 6,55 6,48</td></tr><tr><td>Erlösschmälerungen absolut </td><td>(Leerstand, Mietminderung, -verzicht, Betriebskosten)</td><td>TEUR</td><td>-14.374 -13.472</td></tr><tr><td>Erlösschmälerungen aus Leerstand im Verhältnis </td><td>zu Sollmieten Wohnungen</td><td>% 1,9 1,9</td></tr><tr><td>zu Sollmieten Gewerbe</td><td>% 1,9 2,0</td></tr><tr><td>zu Sollmieten sonstige Einheiten</td><td>% 10,4 10,7</td></tr><tr><td>Modernisierungs-/Instandhaltungsausgaben</td><td>EUR/m²/</td><td>Jahr 26,05 20,02</td><td>Jahr 26,05 20,02</td></tr><tr><td>Gemäß Wirtschaftsplanung</td><td>ware im degewo Konzern für Wohnungen eine durchschnittliche Sollmiete </td><td>von 6,55 EUR/m²/Monat prognostiziert und auch erzielt.</td></tr><tr><td>degewo AG</td><td>2023</td><td>2022</td></tr><tr><td>durch</td><td>schnittliche Sollmiete Wohnungen</td><td>EUR/m²/M</td><td>onat 7,40 7,35</td></tr><tr><td>Erlösschmälerungen absolut </td><td>(Leerstand, Mietminderung, -verzicht, Betriebskosten)</td><td>TEUR</td><td>-3.716 -3.266</td></tr><tr><td>Erlösschmälerungen aus Leerstand im Verhältnis </td><td>zu Sollmieten Wohnungen</td><td>% 3,8 3,8</td></tr><tr><td>zu Sollmieten Gewerbe</td><td>% 3,5 3,2</td></tr><tr><td>zu Sollmieten sonstige Einheiten</td><td>% 12,0 13,9</td></tr><tr><td>Modernisierungs-/Instandhaltungsausgaben</td><td>EUR/m²/</td><td>Jahr 27,43 19,85</td><td>Jah 27,43 19,85</td></tr><tr><td>Anlage II</td><td>Seite 8</td></tr></table>]

In [13]:
import os
from pypdf import PdfReader
import pandas as pd
import re

# Define the search term and the directory
search_terms = [
    ("Aktiva", r"20\d{2}"), 
    ("Passiva", r"20\d{2}"), 
    ("Gewinn", "Verlust", "Rechnung", r"20\d{2}")
]
base_dir = "../Geschaeftsberichte/"

ocr_required = [
    "Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf",
    "Geschaeftsberichte/degewo AG/2018_degewo_Konzernlagebericht_und_Konzernabschluss_fuer_das_Geschaeftsjahr.pdf",
    "Geschaeftsberichte/degewo AG/2019_degewo_Konzernbericht_und_Gleicher_Lohn_fuer_gleiche_Arbeit.pdf",
]

# Initialize a list to store results
results = []

# Recursively search through the directory
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".pdf"):
            filepath = os.path.join(root, file)
            print(filepath)
            if filepath == "../Geschaeftsberichte/bb_2024_gesamt.pdf":
                continue
            if filepath[3:] in ocr_required:
                print(f"Skipping OCR required file: {filepath}")
                continue
            
            try:
                # Open the PDF file
                reader = PdfReader(filepath)
                for page_num, page in enumerate(reader.pages):
                    text = page.extract_text()
                    for terms in search_terms:
                        # Check if all terms in the tuple are found on the page
                        if all(
                            re.search(term, text, re.IGNORECASE) if isinstance(term, str) else False
                            for term in terms
                        ):
                            match = re.search(terms[0], text, re.IGNORECASE)
                            if match:
                                start_idx = match.start()
                                context = text[max(0, start_idx - 50):start_idx + 50]
                                results.append({
                                    "filepath": filepath,
                                    "page": page_num + 1,
                                    "type": terms[0],
                                    "context": context.strip()
                                })
            except Exception as e:
                print(f"Error reading {filepath}: {e}")

# Convert results to a DataFrame
df = pd.DataFrame(results, columns=["filepath", "filename", "page", "context"])

# Save the results to a CSV file
output_csv = "bilanz_search_results.csv"
df.to_csv(output_csv, index=False)

print(f"Search completed. Results saved to {output_csv}.")

../Geschaeftsberichte/bb_2024_gesamt.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2023_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2020.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2020_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2023.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2021_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2019.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2022_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2019_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2021.pdf
../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2017.pdf
../Geschaeftsberichte/IBB/ibb-jahresabschluss-und-lagebericht-2023.pdf
../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2020.pdf
../Geschaeftsberichte/IBB/ibb_geschaeftsberic